<a href="https://colab.research.google.com/github/CleanPegasus/Anime_generation_with_DCGAN/blob/master/Anime_Character_Generation_with_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive

     |████████████████████████████████| 993kB 51.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '0B4wZXrs0DHMHMEl1ODVpMjRTWEk'})
download.GetContentFile('anime-faces.tar.gz')

In [26]:
ls

adc.json  anime-faces/  anime-faces.tar.gz  sample_data/


In [0]:
!tar -xvzf anime-faces.tar.gz

In [42]:
cd anime-faces

/content/anime-faces


In [0]:
!rm config.json annotations.csv

In [22]:
!rm -v ._.DS_Store .DS_Store

removed '._.DS_Store'


In [0]:
ls

In [48]:
cd ..

/content


In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

import tensorflow as tf
from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU, Lambda, K, Conv2DTranspose, Activation, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras_preprocessing import image
from keras.models import load_model, Sequential


In [49]:
src= 'anime-faces'
image_data = []
for file in os.listdir(src):
  file_path = os.path.join(src, file)
  for image in os.listdir(file_path):
    if not image.startswith('.'):
      img_path = os.path.join(file_path, image)    
      #print(img_path)
      img = cv2.imread(img_path)
      print(img.shape)
      image_data.append(img)

(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96, 96, 3)
(96,

In [50]:
image_data = np.asarray(image_data)
image_data.shape

(115085, 96, 96, 3)

In [0]:
def build_gen():
  
  gen_model = Sequential()
  gen_model.add(Dense(units = 2048), activation = 'tanh')
  gen_model.add(Dense(246 * 8 * 8))
  gen_model.add(BatchNormalization())
  gen_model.add(Activation('tanh'))
  gen_model.add(Reshape((8, 8, 256), input_shape = (256 * 8 * 8)))
  gen_model.add(UpSampling2D(size = (2, 2)))
  